In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import networkx as nx
dataFrame = pd.read_csv("/content/drive/MyDrive/GCN Classification/citation_sentiment_corpus.txt", sep = "	", header = None)
dataFrame.columns = ["Source_PaperID", "Target_PaperID", "Sentiment", "Citation_text"]
dataFrame.Sentiment = dataFrame.Sentiment.replace({"o": 1,"p": 2,"n": 0})

In [3]:
Source = dataFrame['Source_PaperID']
Target = dataFrame['Target_PaperID']
Sentiment = dataFrame['Sentiment']
G = nx.DiGraph(directed=True)
edges = pd.DataFrame(
    {
        "source": Source,
        "target": Target,
    }
)
G = nx.from_pandas_edgelist(edges,create_using=nx.DiGraph())

In [4]:
leaderboard = {}
for x in G.nodes:
 leaderboard[x] = len(G[x])
s = pd.Series(leaderboard, name='Citations')
citation_counts = s.to_frame().sort_values('Citations', ascending=False)
citation_counts.value_counts()

Citations
1            1785
2             703
3             295
4             121
0              77
5              59
6              19
7               7
8               3
dtype: int64

In [5]:
citation_counts = citation_counts.reset_index(level=0)
citation_counts.columns = ['Node', 'Citations']
citation_counts.head()

,Node,Citations
0,W08-0306,8
1,N09-1058,8
2,D07-1070,8
3,N09-1049,7
4,P08-1068,7


In [6]:
zero_list = []
for i,j in zip(citation_counts['Node'], citation_counts['Citations']):
    if(j == 0):
      
        zero_list.append(i)
G.remove_nodes_from(zero_list)

In [7]:
len(G.nodes())

2992

In [8]:
import random
from tqdm import tqdm


def parallel_generate_walks(d_graph: dict, global_walk_length: int, num_walks: int, cpu_num: int,
                            sampling_strategy: dict = None, num_walks_key: str = None, walk_length_key: str = None,
                            neighbors_key: str = None, probabilities_key: str = None, first_travel_key: str = None,
                            quiet: bool = False) -> list:
    """
    Generates the random walks which will be used as the skip-gram input.
    :return: List of walks. Each walk is a list of nodes.
    """

    walks = list()

    if not quiet:
        pbar = tqdm(total=num_walks, desc='Generating walks (CPU: {})'.format(cpu_num))

    for n_walk in range(num_walks):

        # Update progress bar
        if not quiet:
            pbar.update(1)

        # Shuffle the nodes
        shuffled_nodes = list(d_graph.keys())
        random.shuffle(shuffled_nodes)

        # Start a random walk from every node
        for source in shuffled_nodes:

            # Skip nodes with specific num_walks
            if source in sampling_strategy and \
                    num_walks_key in sampling_strategy[source] and \
                    sampling_strategy[source][num_walks_key] <= n_walk:
                continue

            # Start walk
            walk = [source]

            # Calculate walk length
            if source in sampling_strategy:
                walk_length = sampling_strategy[source].get(walk_length_key, global_walk_length)
            else:
                walk_length = global_walk_length

            # Perform walk
            while len(walk) < walk_length:

                walk_options = d_graph[walk[-1]].get(neighbors_key, None)

                # Skip dead end nodes
                if not walk_options:
                    break

                if len(walk) == 1:  # For the first step
                    probabilities = d_graph[walk[-1]][first_travel_key]
                    walk_to = random.choices(walk_options, weights=probabilities)[0]
                else:
                    probabilities = d_graph[walk[-1]][probabilities_key][walk[-2]]
                    walk_to = random.choices(walk_options, weights=probabilities)[0]

                walk.append(walk_to)

            walk = list(map(str, walk))  # Convert all to strings

            walks.append(walk)

    if not quiet:
        pbar.close()

    return walks

In [9]:
import os
import random
from collections import defaultdict

import gensim
import networkx as nx
import numpy as np
import pkg_resources
from joblib import Parallel, delayed
from tqdm.auto import tqdm



class Node2Vec:
    FIRST_TRAVEL_KEY = 'first_travel_key'
    PROBABILITIES_KEY = 'probabilities'
    NEIGHBORS_KEY = 'neighbors'
    WEIGHT_KEY = 'weight'
    NUM_WALKS_KEY = 'num_walks'
    WALK_LENGTH_KEY = 'walk_length'
    P_KEY = 'p'
    Q_KEY = 'q'

    def __init__(self, graph: nx.Graph, dimensions: int = 128, walk_length: int = 80, num_walks: int = 10, p: float = 1,
                 q: float = 1, weight_key: str = 'weight', workers: int = 1, sampling_strategy: dict = None,
                 quiet: bool = False, temp_folder: str = None, seed: int = None):
        """
        Initiates the Node2Vec object, precomputes walking probabilities and generates the walks.
        :param graph: Input graph
        :param dimensions: Embedding dimensions (default: 128)
        :param walk_length: Number of nodes in each walk (default: 80)
        :param num_walks: Number of walks per node (default: 10)
        :param p: Return hyper parameter (default: 1)
        :param q: Inout parameter (default: 1)
        :param weight_key: On weighted graphs, this is the key for the weight attribute (default: 'weight')
        :param workers: Number of workers for parallel execution (default: 1)
        :param sampling_strategy: Node specific sampling strategies, supports setting node specific 'q', 'p', 'num_walks' and 'walk_length'.
        :param seed: Seed for the random number generator.
        Use these keys exactly. If not set, will use the global ones which were passed on the object initialization
        :param temp_folder: Path to folder with enough space to hold the memory map of self.d_graph (for big graphs); to be passed joblib.Parallel.temp_folder
        """

        self.graph = graph
        self.dimensions = dimensions
        self.walk_length = walk_length
        self.num_walks = num_walks
        self.p = p
        self.q = q
        self.weight_key = weight_key
        self.workers = workers
        self.quiet = quiet
        self.d_graph = defaultdict(dict)
    
        if sampling_strategy is None:
            self.sampling_strategy = {}
        else:
            self.sampling_strategy = sampling_strategy

        self.temp_folder, self.require = None, None
        if temp_folder:
            if not os.path.isdir(temp_folder):
                raise NotADirectoryError("temp_folder does not exist or is not a directory. ({})".format(temp_folder))

            self.temp_folder = temp_folder
            self.require = "sharedmem"

        if seed is not None:
            random.seed(seed)
            np.random.seed(seed)

        self._precompute_probabilities()
        self.walks = self._generate_walks()

    def _precompute_probabilities(self):
        """
        Precomputes transition probabilities for each node.
        """

        d_graph = self.d_graph
        print(d_graph)
        nodes_generator = self.graph.nodes() if self.quiet \
            else tqdm(self.graph.nodes(), desc='Computing transition probabilities')
        
        for source in tqdm(nodes_generator,desc='Computing transition probabilities'):

            # Init probabilities dict for first travel
            if self.PROBABILITIES_KEY not in d_graph[source]:
                d_graph[source][self.PROBABILITIES_KEY] = dict()

            for current_node in self.graph.neighbors(source):

                # Init probabilities dict
                if self.PROBABILITIES_KEY not in d_graph[current_node]:
                    d_graph[current_node][self.PROBABILITIES_KEY] = dict()

                unnormalized_weights = list()
                d_neighbors = list()

                # Calculate unnormalized weights
                for destination in self.graph.neighbors(current_node):

                    p = self.sampling_strategy[current_node].get(self.P_KEY,
                                                                 self.p) if current_node in self.sampling_strategy else self.p
                    q = self.sampling_strategy[current_node].get(self.Q_KEY,
                                                                 self.q) if current_node in self.sampling_strategy else self.q

                    try:
                        if self.graph[current_node][destination].get(self.weight_key):
                            weight = self.graph[current_node][destination].get(self.weight_key)
                        else: 
                            ## Example : AtlasView({0: {'type': 1, 'weight':0.1}})- when we have edge weight
                            edge = list(self.graph[current_node][destination])[-1]
                            weight = self.graph[current_node][destination][edge].get(self.weight_key)
                            
                    except:
                        weight = 1 
                    
                    if destination == source:  # Backwards probability
                        ss_weight = weight * 1 / p
                    elif destination in self.graph[source]:  # If the neighbor is connected to the source
                        ss_weight = weight
                    else:
                        ss_weight = weight * 1 / q

                    # Assign the unnormalized sampling strategy weight, normalize during random walk
                    unnormalized_weights.append(ss_weight)
                    d_neighbors.append(destination)

                # Normalize
                unnormalized_weights = np.array(unnormalized_weights)
                d_graph[current_node][self.PROBABILITIES_KEY][
                    source] = unnormalized_weights / unnormalized_weights.sum()

            # Calculate first_travel weights for source
            first_travel_weights = []

            for destination in self.graph.neighbors(source):
                first_travel_weights.append(self.graph[source][destination].get(self.weight_key, 1))

            first_travel_weights = np.array(first_travel_weights)
            d_graph[source][self.FIRST_TRAVEL_KEY] = first_travel_weights / first_travel_weights.sum()

            # Save neighbors
            d_graph[source][self.NEIGHBORS_KEY] = list(self.graph.neighbors(source))

    def _generate_walks(self) -> list:
        """
        Generates the random walks which will be used as the skip-gram input.
        :return: List of walks. Each walk is a list of nodes.
        """

        flatten = lambda l: [item for sublist in l for item in sublist]

        # Split num_walks for each worker
        num_walks_lists = np.array_split(range(self.num_walks), self.workers)

        walk_results = Parallel(n_jobs=self.workers, temp_folder=self.temp_folder, require=self.require)(
            delayed(parallel_generate_walks)(self.d_graph,
                                             self.walk_length,
                                             len(num_walks),
                                             idx,
                                             self.sampling_strategy,
                                             self.NUM_WALKS_KEY,
                                             self.WALK_LENGTH_KEY,
                                             self.NEIGHBORS_KEY,
                                             self.PROBABILITIES_KEY,
                                             self.FIRST_TRAVEL_KEY,
                                             self.quiet) for
            idx, num_walks
            in enumerate(num_walks_lists, 1))

        walks = flatten(walk_results)

        return walks

    def fit(self, **skip_gram_params) -> gensim.models.Word2Vec:
        """
        Creates the embeddings using gensim's Word2Vec.
        :param skip_gram_params: Parameters for gensim.models.Word2Vec - do not supply 'size' / 'vector_size' it is
            taken from the Node2Vec 'dimensions' parameter
        :type skip_gram_params: dict
        :return: A gensim word2vec model
        """

        if 'workers' not in skip_gram_params:
            skip_gram_params['workers'] = self.workers

        # Figure out gensim version, naming of output dimensions changed from size to vector_size in v4.0.0
        gensim_version = pkg_resources.get_distribution("gensim").version
        size = 'size' if gensim_version < '4.0.0' else 'vector_size'
        if size not in skip_gram_params:
            skip_gram_params[size] = self.dimensions

        if 'sg' not in skip_gram_params:
            skip_gram_params['sg'] = 1

        return gensim.models.Word2Vec(self.walks, **skip_gram_params)

In [10]:


# FILES

EMBEDDING_FILENAME = '/content/drive/MyDrive/GCN Classification/node2vec_embeddings.txt'

# Create a graph
# graph = nx.fast_gnp_random_graph(n=10000, p=0.5)

# Precompute probabilities and generate walks
node2vec = Node2Vec(G, dimensions=128, walk_length=100, num_walks=200, workers=4)

## if d_graph is big enough to fit in the memory, pass temp_folder which has enough disk space
# Note: It will trigger "sharedmem" in Parallel, which will be slow on smaller graphs
#node2vec = Node2Vec(graph, dimensions=64, walk_length=30, num_walks=200, workers=4, temp_folder="/mnt/tmp_data")

# Embed
model = node2vec.fit(window=10, min_count=1, batch_words=4)  # Any keywords acceptable by gensim.Word2Vec can be passed, `dimensions` and `workers` are automatically passed (from the Node2Vec constructor)

embeddings = np.array([model.wv[str(node)] for node in G.nodes()])

print(embeddings.shape)

defaultdict(<class 'dict'>, {})


Computing transition probabilities:   0%|          | 0/2992 [00:00<?, ?it/s]

Computing transition probabilities:   0%|          | 0/2992 [00:00<?, ?it/s]

(2992, 128)


In [13]:
import os
all_data = []

for root,dirs,files in os.walk('.'):
    for file in files:
        if 'node2vec_embeddings.txt' in file:
            with open(os.path.join(root,file),'r') as f:
                all_data.extend(f.read().splitlines())
all_data

#parse the data
nodes = []
node2vec_cc = []

for i,data in enumerate(all_data):
    elements = data.split(' ')
    node2vec_cc.append(elements[1:])
    nodes.append(elements[0])

In [14]:
df = dataFrame
labels = []
for i in nodes:
    for j,k in zip(df['Source_PaperID'],df['Sentiment']):
        if(i==j):
            labels.append(k)
            break

In [15]:
X = embeddings

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, labels, test_size=0.3, random_state=42)

In [17]:
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score
import numpy as np
from sklearn.model_selection import train_test_split
from imblearn import under_sampling 
from imblearn import over_sampling
from imblearn.over_sampling import SMOTE
import collections, numpy
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn import tree
from sklearn import svm
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.linear_model import SGDClassifier
import matplotlib.pyplot as plt

lr = LogisticRegression(solver='lbfgs', max_iter=1000)
lr.fit(X_train, y_train)
y_pred_lr = lr.predict(X_test)
acc_lr = round(accuracy_score(y_test, y_pred_lr) * 100, 2)
pre_score_lr = round(precision_score(y_test, y_pred_lr, average='weighted', zero_division=1) * 100, 2)
recall_score_lr = round(recall_score(y_test, y_pred_lr, average='weighted') * 100, 2)
fl_score_lr = round(f1_score(y_test, y_pred_lr, average='weighted') * 100, 2)
print(acc_lr)
print(pre_score_lr)
print(recall_score_lr)
print(fl_score_lr)

85.52
76.81
85.52
79.36


In [18]:
DTclf = tree.DecisionTreeClassifier()
DTclf.fit(X_train, y_train)
y_pred_dt = DTclf.predict(X_test) 
acc_dt = round(accuracy_score(y_test, y_pred_dt) * 100,2)
pre_score_dt = round(precision_score(y_test, y_pred_dt, average='weighted', zero_division=1) * 100,2)
recall_score_dt = round(recall_score(y_test, y_pred_dt, average='weighted') * 100,2)
fl_score_dt = round(f1_score(y_test, y_pred_dt, average='weighted') * 100,2)
print(acc_dt)
print(pre_score_dt)
print(recall_score_dt)
print(fl_score_dt)

74.5
75.21
74.5
74.85


In [19]:
RFclf = RandomForestClassifier()
RFclf.fit(X_train, y_train)
y_pred_RF = RFclf.predict(X_test)
acc_rf = round(accuracy_score(y_test, y_pred_RF) * 100,2)
pre_score_rf = round(precision_score(y_test, y_pred_RF, average='weighted', zero_division=1) * 100,2)
recall_score_rf = round(recall_score(y_test, y_pred_RF, average='weighted') * 100,2)
fl_score_rf = round(f1_score(y_test, y_pred_RF, average='weighted') * 100,2)
print(acc_rf)
print(pre_score_rf)
print(recall_score_rf)
print(fl_score_rf)

85.52
76.81
85.52
79.36


In [20]:
SVMclf = svm.SVC()
SVMclf.fit(X_train, y_train)
y_pred_svm = SVMclf.predict(X_test)
acc_svm = round(accuracy_score(y_test, y_pred_svm) * 100,2)
pre_score_svm = round(precision_score(y_test, y_pred_svm, average='weighted', zero_division=1) * 100,2)
recall_score_svm = round(recall_score(y_test, y_pred_svm, average='weighted') * 100,2)
fl_score_svm = round(f1_score(y_test, y_pred_svm, average='weighted') * 100,2)
print(acc_svm)
print(pre_score_svm)
print(recall_score_svm)
print(fl_score_svm)

86.08
88.02
86.08
79.64


In [21]:
ADAclf = AdaBoostClassifier(n_estimators=100, random_state=0)
ADAclf.fit(X_train, y_train)
y_pred_ada = ADAclf.predict(X_test)
acc_ada = round(accuracy_score(y_test, y_pred_ada) * 100,2)
pre_score_ada = round(precision_score(y_test, y_pred_ada, average='weighted', zero_division=1) * 100,2)
recall_score_ada = round(recall_score(y_test, y_pred_ada, average='weighted') * 100,2)
fl_score_ada = round(f1_score(y_test, y_pred_ada, average='weighted') * 100,2)
print(acc_ada)
print(pre_score_ada)
print(recall_score_ada)
print(fl_score_ada)

84.97
75.9
84.97
79.51


In [22]:
ETclf = ExtraTreesClassifier(n_estimators=100, random_state=0)
ETclf.fit(X_train, y_train)
y_pred_et = ETclf.predict(X_test)
acc_et = round(accuracy_score(y_test, y_pred_et) * 100,2)
pre_score_et = round(precision_score(y_test, y_pred_et, average='weighted', zero_division=1) * 100,2)
recall_score_et = round(recall_score(y_test, y_pred_et, average='weighted') * 100,2)
fl_score_et = round(f1_score(y_test, y_pred_et, average='weighted') * 100,2)
print(acc_et)
print(pre_score_et)
print(recall_score_et)
print(fl_score_et)

85.19
73.99
85.19
79.2


In [23]:
SGDclf = SGDClassifier()
SGDclf.fit(X_train, y_train)
y_pred_sgd = SGDclf.predict(X_test)
acc_sgd = round(accuracy_score(y_test, y_pred_sgd) * 100,2)
pre_score_sgd = round(precision_score(y_test, y_pred_sgd, average='weighted', zero_division=1) * 100,2)
recall_score_sgd = round(recall_score(y_test, y_pred_sgd, average='weighted') * 100,2)
fl_score_sgd = round(f1_score(y_test, y_pred_sgd, average='weighted') * 100,2)
print(acc_sgd)
print(pre_score_sgd)
print(recall_score_sgd)
print(fl_score_sgd)

85.75
76.84
85.75
79.47
